In [2]:
# imports 

from util import Eviction_Scraper, normalize_data, initiate_driver
import pandas as pd
import numpy as np
import psycopg2.extras

from server import initiate_db
from db import DB

# initiate conn with database
conn = initiate_db()

# initiate an instance of the DB class
db = DB(conn)

1. Test creating tables in db. Quering a newly created table must return no results:

In [3]:
db.create_schema()
db.query_all_records('eviction_records')

[]

2. Test injecting new records: 

In [4]:
data = pd.read_csv('evictions_2019-2022_colnames_modified.csv')
db.inject_data(data, 'eviction_records')
db.query_all_records('eviction_records')

converted pd into records
finished commit


[{'case_id': '19CV00674',
  'court': 'MUNICIPAL CIVIL',
  'case_caption': 'CHERYL HOFF VS. CLEON LEWIS ET AL',
  'judge': 'UNAVAILABLE',
  'filed_date': datetime.date(2019, 1, 7),
  'case_type': 'G2-EVICTION',
  'amount': 1800,
  'disposition': 'DEFAULT JUDGMENT',
  'disposition_date': datetime.date(2019, 5, 14),
  'plaintiff_name': 'CHERYL HOFF',
  'plaintiff_address': '1505 N PARK AVE CINCINNATI OH 45215',
  'plaintiff_attorney': 'RICHARDS/STUART/L',
  'defendant_name': 'CLEON LEWIS',
  'defendant_address': '1323 WOODLAND AVE CINCINNATI OH 45237',
  'defendant_attorney': None,
  'last_updated': datetime.date(2022, 7, 28)},
 {'case_id': '19CV00673',
  'court': 'MUNICIPAL CIVIL',
  'case_caption': 'BERT YOUKEY TRUST VS. MICHAEL STURGEON ET AL',
  'judge': 'UNAVAILABLE',
  'filed_date': datetime.date(2019, 1, 7),
  'case_type': 'G2-EVICTION',
  'amount': 2150,
  'disposition': 'JUDGMENT FOR PLAINTIFF',
  'disposition_date': datetime.date(2019, 7, 15),
  'plaintiff_name': 'BERT YOUKEY TR

3. Test web-scraping new records and adding them to our db:
issue 1: chrome browser's version was more recent than chromedriver's. Needed to update.A few more bugs came out of testing. Now it's running successfully.

In [5]:
db.add_new_records(end_date='07152022')

found 141 records
found 363 records
finished scraping. about to inject records into the db
converted pd into records
finished commit


In [8]:
with conn.cursor() as c:
    c.execute('select * from eviction_records order by filed_date DESC limit 3;')
    data = c.fetchall()

data

[('22CV13831',
  'MUNICIPAL CIVIL',
  'SANCTUARY HOLDING GROUP LLC VS. JOSH GRATHWOHL',
  'UNAVAILABLE',
  datetime.date(2022, 7, 15),
  'G1-EVICTION',
  None,
  'JUDGMENT FOR PLAINTIFF',
  datetime.date(2022, 8, 17),
  'SANCTUARY HOLDING GROUP LLC',
  '%3BLONDES REALTY LLC 5981 HARRISON AVE SUITE #4CINCINNATI OH 45248',
  'HASSMAN/ROBERT/C/JR',
  'JOSH GRATHWOHL',
  '3313 NORTH BEND RD UNIT I CINCINNATI OH 45239',
  None,
  datetime.date(2022, 9, 18)),
 ('22CV13830',
  'MUNICIPAL CIVIL',
  '3801 DINA LLC VS. JASMINE HENDERSON',
  'UNAVAILABLE',
  datetime.date(2022, 7, 15),
  'G2-EVICTION',
  2908,
  None,
  None,
  '3801 DINA LLC',
  '8221 CORNELL RD STE 410 CINCINNATI OH 45249',
  'GLENNON/THOMAS',
  'JASMINE HENDERSON',
  '3811 DINA AVE APT 9 CINCINNATI OH 45211',
  None,
  datetime.date(2022, 9, 18)),
 ('22CV13829',
  'MUNICIPAL CIVIL',
  '3801 DINA LLC VS. SETH Y KUTANDO',
  'UNAVAILABLE',
  datetime.date(2022, 7, 15),
  'G2-EVICTION',
  1710,
  'DISMISSED',
  datetime.date(2022,

4. Testing update of records method. After lenghty debugging, it works: 

In [7]:
db.update_existing_records_disposition(records_per_batch=30, limit=10)

     case_id disposition disposition_date last_updated
0  19CV01204        None             None   2022-09-18
1  19CV01199        None             None   2022-09-18
2  19CV01195        None             None   2022-09-18
3  19CV01194        None             None   2022-09-18
4  19CV01193        None             None   2022-09-18
5  19CV01186        None             None   2022-09-18
6  19CV01185        None             None   2022-09-18
7  19CV01173        None             None   2022-09-18
8  19CV01172        None             None   2022-09-18
9  19CV01009        None             None   2022-09-18


In [4]:
with conn.cursor() as c:
    c.execute(
        """select * from eviction_records where case_id in (
            '19CV01186', '19CV01185', '19CV01173', '19CV01172');""")
    data = c.fetchall()

data

[('19CV01172',
  'MUNICIPAL CIVIL',
  'CMHA VS. WALTER ROGERS',
  'UNAVAILABLE',
  datetime.date(2019, 1, 14),
  'G2-EVICTION',
  699,
  None,
  None,
  'CINCINNATI METRO HOUSING AUTH',
  '1627 WESTERN AVENUE CINCINNATI OH 45214',
  'JOHNSON/DONTE',
  'WALTER E ROGERS',
  '330 FOREST AVE SUITE 1303 CINCINNATI OH 452292462',
  None,
  datetime.date(2022, 9, 18)),
 ('19CV01173',
  'MUNICIPAL CIVIL',
  'CMHA VS. BONNIE ROGESKI',
  'UNAVAILABLE',
  datetime.date(2019, 1, 14),
  'G2-EVICTION',
  607,
  None,
  None,
  'CINCINNATI METRO HOUSING AUTH',
  '1627 WESTERN AVENUE CINCINNATI OH 45214',
  'JOHNSON/DONTE',
  'BONNIE E ROGESKI',
  '1820 RUTLAND AVE SUITE 106 CINCINNATI OH 452071244',
  None,
  datetime.date(2022, 9, 18)),
 ('19CV01185',
  'MUNICIPAL CIVIL',
  'CMHA VS. MAKAILA LUCKEY',
  'UNAVAILABLE',
  datetime.date(2019, 1, 14),
  'G2-EVICTION',
  393,
  None,
  None,
  'CINCINNATI METRO HOUSING AUTH',
  '1627 WESTERN AVENUE CINCINNATI OH 45214',
  'JOHNSON/DONTE',
  'MAKAILA A LUC